# Hierarchical Indices in Document Retrieval

## Overview
This code implements a Hierarchical Indexing system for document retrieval, utilizing two levels of encoding: document-level summaries and detailed chunks. This approach aims to improve the efficiency and relevance of information retrieval by first identifying relevant document sections through summaries, then drilling down to specific details within those sections.

## Motivation
Traditional flat indexing methods can struggle with large documents or corpus, potentially missing context or returning irrelevant information. Hierarchical indexing addresses this by creating a two-tier search system, allowing for more efficient and context-aware retrieval.

## Key Components
- **PDF processing and text chunking**
- **Asynchronous document summarization using OpenAI's GPT-4**
- **Vector store creation for both summaries and detailed chunks using FAISS and OpenAI embeddings**
- **Custom hierarchical retrieval function**

## Method Details

### Document Preprocessing and Encoding
1. The PDF is loaded and split into documents (likely by page).
2. Each document is summarized asynchronously using GPT-4.
3. The original documents are also split into smaller, detailed chunks.
4. Two separate vector stores are created:
   - One for document-level summaries
   - One for detailed chunks

### Asynchronous Processing and Rate Limiting
- The code uses asynchronous programming (`asyncio`) to improve efficiency.
- Implements batching and exponential backoff to handle API rate limits.

### Hierarchical Retrieval
The `retrieve_hierarchical` function implements the two-tier search:
1. It first searches the summary vector store to identify relevant document sections.
2. For each relevant summary, it then searches the detailed chunk vector store, filtering by the corresponding page number.
3. This approach ensures that detailed information is retrieved only from the most relevant document sections.

## Benefits of this Approach
- **Improved Retrieval Efficiency:** By first searching summaries, the system can quickly identify relevant document sections without processing all detailed chunks.
- **Better Context Preservation:** The hierarchical approach helps maintain the broader context of retrieved information.
- **Scalability:** This method is particularly beneficial for large documents or corpus, where flat searching might be inefficient or miss important context.
- **Flexibility:** The system allows for adjusting the number of summaries and chunks retrieved, enabling fine-tuning for different use cases.

## Implementation Details
- **Asynchronous Programming:** Utilizes Python's `asyncio` for efficient I/O operations and API calls.
- **Rate Limit Handling:** Implements batching and exponential backoff to manage API rate limits effectively.
- **Persistent Storage:** Saves the generated vector stores locally to avoid unnecessary recomputation.

## Conclusion
Hierarchical indexing represents a sophisticated approach to document retrieval, particularly suitable for large or complex document sets. By leveraging both high-level summaries and detailed chunks, it offers a balance between broad context understanding and specific information retrieval. This method has potential applications in various fields requiring efficient and context-aware information retrieval, such as legal document analysis, academic research, or large-scale content management systems.


<img src="../img/hierarchical_indices.svg" alt="title" width="1000" height="100%"/>


In [6]:
import asyncio
import os
import sys
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.chains.summarize.chain import load_summarize_chain
from langchain.docstore.document import Document
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
# from langchain.vectorstores import FAISS
from langchain_pinecone import PineconeVectorStore


/home/geetendra/Desktop/learning/learning-ai/lib/python3.12/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [8]:
import os

os.environ["OPENAI_API_KEY"] = ""
os.environ["PINECONE_API_KEY"] = ""

In [9]:
path = "../data/Understanding_Climate_Change.pdf"


In [10]:
## Helper functions
import random
from openai import RateLimitError

async def exponential_backoff(attempt):
    """
    Implements exponential backoff with a jitter.
    
    Args:
        attempt: The current retry attempt number.
        
    Waits for a period of time before retrying the operation.
    The wait time is calculated as (2^attempt) + a random fraction of a second.
    """
    # Calculate the wait time with exponential backoff and jitter
    wait_time = (2 ** attempt) + random.uniform(0, 1)
    print(f"Rate limit hit. Retrying in {wait_time:.2f} seconds...")

    # Asynchronously sleep for the calculated wait time
    await asyncio.sleep(wait_time)



async def retry_with_exponential_backoff(coroutine, max_retries=5):
    """
    Retries a coroutine using exponential backoff upon encountering a RateLimitError.
    
    Args:
        coroutine: The coroutine to be executed.
        max_retries: The maximum number of retry attempts.
        
    Returns:
        The result of the coroutine if successful.
        
    Raises:
        The last encountered exception if all retry attempts fail.
    """
    for attempt in range(max_retries):
        try:
            # Attempt to execute the coroutine
            return await coroutine
        except RateLimitError as e:
            # If the last attempt also fails, raise the exception
            if attempt == max_retries - 1:
                raise e

            # Wait for an exponential backoff period before retrying
            await exponential_backoff(attempt)

    # If max retries are reached without success, raise an exception
    raise Exception("Max retries reached")

In [11]:
async def encode_pdf_hierarchical(path, chunk_size=1000, chunk_overlap=200, is_string=False):
    """
    Asynchronously encodes a PDF book into a hierarchical vector store using OpenAI embeddings.
    Includes rate limit handling with exponential backoff.
    
    Args:
        path: The path to the PDF file.
        chunk_size: The desired size of each text chunk.
        chunk_overlap: The amount of overlap between consecutive chunks.
        
    Returns:
        A tuple containing two FAISS vector stores:
        1. Document-level summaries
        2. Detailed chunks
    """
    # Load the PDF file
    loader = PyPDFLoader(path)
    documents = await asyncio.to_thread(loader.load)
    print(f"Loaded {documents} documents.")

        
    summary_llm = ChatOpenAI(temperature=0, model_name="gpt-4o-mini", max_tokens=4000)
    summary_chain = load_summarize_chain(summary_llm, chain_type="map_reduce")

    async def summarize_doc(doc):
        """
        Summarizes a single document with rate limit handling.
        
        Args:
            doc: The document to be summarized.
            
        Returns:
            A summarized Document object.
        """
        # Retry the summarization with exponential backoff
        summary_output = await retry_with_exponential_backoff(summary_chain.ainvoke([doc]))
        summary = summary_output['output_text']
        return Document(
            page_content=summary,
            metadata={"source": path, "page": doc.metadata["page"], "summary": True}
        )
    # Process documents in smaller batches to avoid rate limits
    batch_size = 5  # Adjust this based on your rate limits
    summaries = []
    for i in range(0, len(documents), batch_size):
        batch = documents[i:i+batch_size]
        batch_summaries = await asyncio.gather(*[summarize_doc(doc) for doc in batch])
        summaries.extend(batch_summaries)
        await asyncio.sleep(1) 

    print("summaries", summaries)


    # Split documents into detailed chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, chunk_overlap=chunk_overlap, length_function=len
    )
    detailed_chunks = await asyncio.to_thread(text_splitter.split_documents, documents)

    # Update metadata for detailed chunks
    for i, chunk in enumerate(detailed_chunks):
        chunk.metadata.update({
            "chunk_id": i,
            "summary": False,
            "page": int(chunk.metadata.get("page", 0))
        })

    print("detailed_chunks", detailed_chunks)
    
    # async def create_vectorstore(docs):
    #     """
    #     Creates a vector store from a list of documents with rate limit handling.
        
    #     Args:
    #         docs: The list of documents to be embedded.
            
    #     Returns:
    #         A FAISS vector store containing the embedded documents.
    #     """
    #     return await retry_with_exponential_backoff(
    #         asyncio.to_thread(FAISS.from_documents, docs, embeddings)
    #     )

    # # Generate vector stores for summaries and detailed chunks concurrently
    # summary_vectorstore, detailed_vectorstore = await asyncio.gather(
    #     create_vectorstore(summaries),
    #     create_vectorstore(detailed_chunks)
    # )

    return summaries, detailed_chunks

In [12]:
summaries_data, detailed_chunks_data = await encode_pdf_hierarchical(path)


Loaded [Document(metadata={'source': '../data/Understanding_Climate_Change.pdf', 'page': 0}, page_content='Understanding Climate Change  \nChapter 1: Introduction to Climate Change  \nClimate change refers to significant, long -term changes in the global climate. The term \n"global climate" encompasses the planet\'s overall weather patterns, including temperature, \nprecipitation, and wind patterns, over an extended period. Over the past cent ury, human \nactivities, particularly the burning of fossil fuels and deforestation, have significantly \ncontributed to climate change.  \nHistorical Context  \nThe Earth\'s climate has changed throughout history. Over the past 650,000 years, there have \nbeen seven cycles of glacial advance and retreat, with the abrupt end of the last ice age about \n11,700 years ago marking the beginning of the modern climate era and  human civilization. \nMost of these climate changes are attributed to very small variations in Earth\'s orbit that \nchange the 

In [15]:
vector_store = PineconeVectorStore(index_name="gen-ai", embedding=OpenAIEmbeddings(model="text-embedding-3-large"))


In [16]:
vector_store.add_documents(summaries_data, namespace="hierarchical_indexing_summary_vectors")
vector_store.add_documents(detailed_chunks_data, namespace="hierarchical_indexing_detailed_documents_vectors")

['660b254e-8ed0-4aa3-8a5b-8099ef45b326',
 'd298a964-44a0-4a99-b5ff-12b6260b62d0',
 '555847ad-e389-434e-83d4-f8e1a00c328b',
 '8504877a-818e-4db7-a309-9bc5e9591a0a',
 '1c9d8ad7-d3d1-4930-99ed-71025a4b910b',
 'fb04e96f-9368-4b6a-b881-5528db2e06a4',
 '9d92ad02-1428-4416-8af0-df03cf047110',
 '7465336a-d8a4-4aa8-96df-8a9021a774fa',
 '7ce47255-b821-46bd-98e7-a9ddcf8e730b',
 '1c968942-1a7f-4adb-b67c-0ce162effbda',
 '08ab6f85-9ad3-4296-84af-c35222075114',
 '45886572-d183-4373-9892-880d061a416d',
 '3fb2dc07-842c-4343-8c2a-d946d6897cd3',
 'cc514bc3-067f-44e4-bde6-38959b1b9746',
 '7dfa5633-5aba-4f34-851a-8b3d0b88c72c',
 '114db85b-6500-414a-a349-689869a2a29a',
 'b870c61a-c39d-4a32-8a5f-b82259d6e359',
 'a69ceabf-cdc5-455b-83ec-819a49d2b5e6',
 'e94a7bd0-2e08-4580-80fa-fe8b78bab628',
 'd0a59073-fc24-4c2a-8f0a-ea042b5851c3',
 'facc4aa8-c04e-49d8-a856-d3dc6d244ee8',
 'b0730d9e-0af6-4efd-90bc-5869e09ea8bb',
 '8c19239a-202b-4b10-8d15-52a404b1294d',
 '4ae5afc3-5900-4c18-bf1c-e6c4a3846836',
 'c4f38960-e9a9-

In [36]:
def retrieve_hierarchical(query, vector_store,  k_summaries=3, k_chunks=5):
    """
    Performs a hierarchical retrieval using the query.

    Args:
        query: The search query.
        summary_vectorstore: The vector store containing document summaries.
        detailed_vectorstore: The vector store containing detailed chunks.
        k_summaries: The number of top summaries to retrieve.
        k_chunks: The number of detailed chunks to retrieve per summary.

    Returns:
        A list of relevant detailed chunks.
    """
    
    # Retrieve top summaries
    top_summaries = vector_store.similarity_search(query, k=k_summaries, namespace="hierarchical_indexing_summary_vectors")
    print(f"Top summaries: {top_summaries}")
    relevant_chunks = []
    for summary in top_summaries:
    #     # For each summary, retrieve relevant detailed chunks
        page_number = summary.metadata["page"]
        page_filter = {"page": page_number}
        print(f"Retrieving chunks for page {page_number}")
        print(f"Filter: {page_filter}")
        page_chunks = vector_store.similarity_search(
            query, 
            k=k_chunks, 
            filter=page_filter,
            namespace="hierarchical_indexing_detailed_documents_vectors"
        )
        relevant_chunks.extend(page_chunks)
    
    return relevant_chunks

In [40]:
query = "What is the greenhouse effect?"
results = retrieve_hierarchical(query, vector_store)
# print(f"Results for query '{results}':")
# Print results
for chunk in results:
    print(f"Score: {chunk}")
    # print(f"Page: {chunk.metadata['page']}")
    # print(f"Content: {chunk.page_content}...")  # Print first 100 characters
    print("---")

Top summaries: [Document(id='bb0fcedf-d06a-4fce-930c-f1a036ae9243', metadata={'page': 0.0, 'source': '../data/Understanding_Climate_Change.pdf', 'summary': True}, page_content='Chapter 1 of "Understanding Climate Change" discusses climate change as significant, long-term shifts in global weather patterns primarily caused by human activities like fossil fuel use and deforestation. It reviews Earth\'s climatic history, highlighting cycles of glacial changes over 650,000 years and the onset of the modern climate era post-ice age. Recent data from the IPCC show alarming trends in temperature, sea levels, and extreme weather linked to human-induced greenhouse gas emissions. Chapter 2 delves into the main causes of climate change, focusing on greenhouse gases—such as carbon dioxide, methane, and nitrous oxide—and the impact of fossil fuel consumption since the industrial revolution.'), Document(id='d3006014-a4c9-4a45-a3f6-106c15307351', metadata={'page': 3.0, 'source': '../data/Understanding